# Imports

In [1]:
import pandas as pd
from google.colab import drive
import os
from ast import literal_eval

# Mount Google Drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/schoolwork/grad/23_spring/CS-263-NLP/final-project/NLP-final-project/implementation/preprocess')

Mounted at /content/drive


# Functions

In [76]:
# convert a token list to a string
def tokens_to_text(token_list):
  output=""
  for token in token_list:
    output += token
  return output

# convert token list to string without uncessary leading chars
def tokens_to_text_without_leading_chars(token_list):
  output=""
  for i, token in enumerate(token_list):
    if i == 0:
      token_parts = token.split()
      if (len(token_parts) > 1):
        for token_part in token_parts[1:]:
          output += token_part
      else:
        output += token.strip()
    else:
      output += token

  return output

# extract the claims from the tokens and the indices
def tokens_to_claim_text(token_list, span_start_index, span_end_index):
  output = []

  for claim_span in zip(span_start_index, span_end_index):
    if (claim_span[0] < claim_span[1]):
      output.append(tokens_to_text_without_leading_chars(token_list[claim_span[0] : claim_span[1]+1]))

  return output

# take a dataframe and convert the column with tokens to a column with tweet strings --> returns the new dataframe
def new_df_with_strings(df):
  tweets = []
  for token_list in df["tokens"].values:
    tweet = tokens_to_text(token_list)
    tweets.append(tweet)

  df["tweets"] = tweets
  df = df[["tweets", "tokens", "claim_label", "span_start_index", "span_end_index"]]

  return df

# take a dataframe and add a column with a list of claims made in the tweet
def new_df_with_claim_text(df):
  claims = []
  for i in range(df.shape[0]):
    claim = tokens_to_claim_text(df["tokens"][i], df["span_start_index"][i], df["span_end_index"][i])
    claims.append(claim)

  df["claims"] = claims

  return df

In [67]:
# read original files
df_train = pd.read_csv("../data/train.csv", converters={'tokens':literal_eval, 'span_start_index':literal_eval, 'span_end_index':literal_eval})
df_dev = pd.read_csv("../data/dev.csv", converters={'tokens':literal_eval, 'span_start_index':literal_eval, 'span_end_index':literal_eval})

In [80]:
# add tweets columns
df_train_with_tweets = new_df_with_strings(df_train)
df_dev_with_tweets = new_df_with_strings(df_dev)

# add claim lists
df_train_with_claims = new_df_with_claim_text(df_train)
df_dev_with_claims = new_df_with_claim_text(df_dev)

# save new df's
df_train_with_claims.to_csv("../data/train-T5.csv", index=False)
df_dev_with_claims.to_csv("../data/dev-T5.csv", index=False)